# Examen 1: NLP

- Néstor I. Martínez Ostoa
- Procesamiento de Lenguaje Natural
- LCD, IIMAS, UNAM
- Octubre 2021

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("spam.csv")
print(df.shape)
print(df["target"].unique())
df.head()

(5572, 2)
['ham' 'spam']


,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


## Preguntas

### 1. ¿Qué porcentaje de los documentos en spam.csv son spam?

In [7]:
num_of_docs = df.shape[0]
num_of_spam_docs = df[df["target"] == "spam"].shape[0]
num_of_ham_docs = df[df["target"] == "ham"].shape[0]

#print(num_of_docs, num_of_spam_docs, num_of_ham_docs)

print((num_of_spam_docs / num_of_docs)*100)

13.406317300789663


### 2. Dados los siguientes vectores, calcule la similitud coseno entre ellos:

- $V_1=[1,0,-1,6,8]$
- $V_2=[0,11,4,7,6]$

In [25]:
def cosine_similarity(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)

    dot_prod_of_vectors = np.dot(v1, v2)
    norm_of_vectors = np.linalg.norm(v1) * np.linalg.norm(v2)

    sim = round(dot_prod_of_vectors / norm_of_vectors, 4)

    return sim

In [26]:
v1 = [1,0,-1,6,8]
v2 = [0,11,4,7,6]
print(cosine_similarity(v1, v2))

0.5715


### 3. ¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

In [45]:
def non_word_char_count(doc):
    count = 0
    for c in doc:
        if c.isalpha() or c.isdigit() or c == "_": 
            continue
        count += 1
    return count

In [48]:
non_word_ham = []
non_word_spam = []

for idx, row in df.iterrows():
    doc = row["text"]
    target = row["target"]

    non_words = non_word_char_count(doc)

    if target == "ham":
        non_word_ham.append(non_words)
    else:
        non_word_spam.append(non_words)

non_word_ham = np.array(non_word_ham)
non_word_spam = np.array(non_word_spam)

print(f"Ham: {np.mean(non_word_ham)}")
print(f"Spam: {np.mean(non_word_spam)}")


Ham: 17.29181347150259
Spam: 29.042838018741634


### 4. ¿Cuál es la longitud promedio de los documentos (número de caracteres) para los documentos spam y no spam?

In [57]:
def get_avg_len(target):
    docs = df[df["target"]==target]["text"].values
    return np.mean([len(d) for d in docs])

sl = get_avg_len("spam")
hl = get_avg_len("ham")

print(f"Spam: {sl}")
print(f"Ham: {hl}")

Spam: 138.8661311914324
Ham: 71.02362694300518


### 5. Observe la siguiente imagen y utilice los vectores de palabras para predecir a cuál país pertenece el río Volga y cuál es la distancia euclidiana 'd' entre su vector de predicción y el vector correcto. Recuerde que V1-R1+R2=P2

In [71]:
def euclidean(A, B):
    diff = A-B
    d = np.sqrt(np.sum(diff*diff))
    return d

In [72]:
r1 = np.array([9,8])
v1 = np.array([4,9])
r2 = np.array([8,1])
p2 = v1-r1+r2

v2 = np.array([2,1])
print(euclidean(v2, p2))

1.4142135623730951


### 6. ¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

In [77]:
def get_avg_digits(docs):
    digits = []
    for doc in docs:
        c = 0
        for d in doc:
            if d.isdigit(): c += 1
        digits.append(c)
    return np.mean(np.array(digits))

In [80]:
spam_docs = df[df["target"]=="spam"]["text"].values
ham_docs = df[df["target"]=="ham"]["text"].values

print(f"Spam: {get_avg_digits(spam_docs)}")
print(f"Ham: {get_avg_digits(ham_docs)}")

Spam: 15.759036144578314
Ham: 0.2992746113989637


### 7. Suponga que en el dataset de spams el 25%  de los email que son spam contiene la palabra “gane”. También se sabe que un total de 13% de emails en el dataset contienen “gane”, y que el 40% del número total de correos son spam. ¿Cuál es la probabilidad de que el siguiente correo sea spam?: “Gane 1,000 pesos enviando la palabra LOTERIA al 3344”

---
Definimos las siguientes variables aleatorias:

- $X$: SPAM
- $Y$: "Gane"

Utilizando el teorema de Bayes: 
$$P(X|Y) = \frac{P(Y|X)*P(X)}{P(Y)}$$

In [82]:
P_YX = 0.25
P_X = 0.4
P_Y = 0.13

P_XY = (P_YX*P_X)/P_Y

print(P_XY)

0.7692307692307693


### 8. Dado el siguiente documento, encontrar el modelo de lenguaje n-grama para n = 3 y encontrar ¿Cuántos 3-gramas de palabras existen en el documento?


Abstract Language users never choose words randomly, and language is essentially non-random. Statistical hypothesis testing uses a null hypothesis, which posits randomness. Hence, when we look at linguistic phenomena in corpora, the null hypothesis will never be true. Moreover, where there is enough data, we shall (almost) always be able to establish.

In [6]:
str_ = "Abstract Language users never choose words randomly, and language is essentially non-random. Statistical hypothesis testing uses a null hypothesis, which posits randomness. Hence, when we look at linguistic phenomena in corpora, the null hypothesis will never be true. Moreover, where there is enough data, we shall (almost) always be able to establish."
str_ = str_.replace(",", "")
str_ = str_.replace("-", "")
str_ = str_.replace(".", "")
str_ = str_.replace("(", "")
str_ = str_.replace(")", "")

doc = str_.split(" ")

trigrams = []

wc = 0
trigram = ""
for word in str_:
    if wc < 3: 
        trigram += word + " "
        wc += 1
    else:
        trigrams.append(trigram)
        trigram = ""
        wc = 0


print(len(trigrams))

85
